In [2]:
os.chdir('..')

In [3]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery
from smallscout.params import *
import db_dtypes

In [4]:
df = pd.read_csv('raw_data/data_for_preprocessing.csv', index_col=0)
df.shape

(170124, 59)

In [5]:
df= pd.read_csv('raw_data/datasets/data_for_preprocessing.csv', index_col=0)
df.shape

(173345, 60)

In [7]:
df['qtr'] = df.quarter.apply(lambda x: x.split('-')[1])
df.head()

,cik,name,date,Assets,AssetsCurrent,Cash,AssetsNoncurrent,Liabilities,LiabilitiesCurrent,LiabilitiesNoncurrent,...,market_cap,mc_qtr_growth,mc_qtr_growth_pct,mc_yr_growth,mc_yr_growth_pct,mc_2yr_growth,mc_2yr_growth_pct,small_cap,micro_cap,qtr
0,1750,AAR CORP,2011-02-28,1.655991e+09,9.278390e+08,54716000.0,409295000.0,8.513950e+08,419182000.0,432213000.0,...,1045.889727,46.783392,0.046825,77.281557,0.079786,NaN,NaN,1,0,Q1
1,1750,AAR CORP,2011-05-31,1.703727e+09,9.139850e+08,57433000.0,465365000.0,8.684380e+08,416010000.0,452428000.0,...,1024.472219,-21.417508,-0.020478,306.796787,0.427487,NaN,NaN,1,0,Q2
2,1750,AAR CORP,2011-08-31,1.752372e+09,9.442470e+08,35523000.0,472856000.0,9.032430e+08,350085000.0,553158000.0,...,882.619592,-141.852627,-0.138464,255.395538,0.407184,NaN,NaN,1,0,Q3
3,1750,AAR CORP,2011-11-30,1.821612e+09,9.550530e+08,27870000.0,521431000.0,9.582200e+08,374944000.0,583276000.0,...,727.886752,-154.732840,-0.175311,-271.219583,-0.271462,NaN,NaN,1,0,Q4
4,1750,AAR CORP,2012-02-29,2.220293e+09,1.065389e+09,59294000.0,797765000.0,1.328974e+09,560986000.0,767988000.0,...,899.522315,171.635564,0.235800,-146.367411,-0.139945,-69.085854,-0.071325,1,0,Q1


In [8]:
df.columns

Index(['cik', 'name', 'date', 'Assets', 'AssetsCurrent', 'Cash',
       'AssetsNoncurrent', 'Liabilities', 'LiabilitiesCurrent',
       'LiabilitiesNoncurrent', 'Equity', 'HolderEquity', 'RetainedEarnings',
       'AdditionalPaidInCapital', 'TreasuryStockValue', 'TemporaryEquity',
       'RedeemableEquity', 'LiabilitiesAndEquity', 'Revenues', 'CostOfRevenue',
       'GrossProfit', 'OperatingExpenses', 'OperatingIncomeLoss',
       'IncomeLossFromContinuingOperationsBeforeIncomeTaxExpenseBenefit',
       'AllIncomeTaxExpenseBenefit', 'IncomeLossFromContinuingOperations',
       'IncomeLossFromDiscontinuedOperationsNetOfTax', 'ProfitLoss',
       'NetIncomeLossAttributableToNoncontrollingInterest', 'NetIncomeLoss',
       'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInOperatingActivities',
     

In [9]:
cols = ['name', 'TICKER', 'date', 'quarter', 'Assets', 'AssetsCurrent', 'Cash', 'AssetsNoncurrent', 'Liabilities', 'LiabilitiesCurrent', 
        'LiabilitiesNoncurrent', 'Equity', 'HolderEquity', 'RetainedEarnings', 'AdditionalPaidInCapital', 
        'TreasuryStockValue', 'TemporaryEquity', 'RedeemableEquity', 'LiabilitiesAndEquity', 'Revenues', 
        'CostOfRevenue', 'GrossProfit', 'OperatingExpenses', 'OperatingIncomeLoss', 
        'IncomeLossFromContinuingOperationsBeforeIncomeTaxExpenseBenefit', 'AllIncomeTaxExpenseBenefit', 
        'IncomeLossFromContinuingOperations', 'IncomeLossFromDiscontinuedOperationsNetOfTax', 'ProfitLoss', 
        'NetIncomeLossAttributableToNoncontrollingInterest', 'NetIncomeLoss', 
        'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations', 
        'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations', 
        'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations', 
        'NetCashProvidedByUsedInOperatingActivities', 'NetCashProvidedByUsedInFinancingActivities', 
        'NetCashProvidedByUsedInInvestingActivities',
       'CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations', 
        'CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations', 
        'CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations', 'EffectOfExchangeRateFinal', 
        'CashPeriodIncreaseDecreaseIncludingExRateEffectFinal', 'afs', 'sic_2d', 'GDP', 'interest_rate', 
        'unemployment_rate', 'median_cpi', 'small_cap', 'micro_cap', 'market_cap', 'qtr']

In [10]:
df = df[cols]

In [11]:
df.columns

Index(['name', 'TICKER', 'date', 'quarter', 'Assets', 'AssetsCurrent', 'Cash',
       'AssetsNoncurrent', 'Liabilities', 'LiabilitiesCurrent',
       'LiabilitiesNoncurrent', 'Equity', 'HolderEquity', 'RetainedEarnings',
       'AdditionalPaidInCapital', 'TreasuryStockValue', 'TemporaryEquity',
       'RedeemableEquity', 'LiabilitiesAndEquity', 'Revenues', 'CostOfRevenue',
       'GrossProfit', 'OperatingExpenses', 'OperatingIncomeLoss',
       'IncomeLossFromContinuingOperationsBeforeIncomeTaxExpenseBenefit',
       'AllIncomeTaxExpenseBenefit', 'IncomeLossFromContinuingOperations',
       'IncomeLossFromDiscontinuedOperationsNetOfTax', 'ProfitLoss',
       'NetIncomeLossAttributableToNoncontrollingInterest', 'NetIncomeLoss',
       'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInOperatingActi

In [12]:
def load_data_to_bq(
        data: pd.DataFrame,
        gcp_project:str,
        bq_dataset:str,
        table: str,
        truncate: bool
    ) -> None:
    """
    - Save the DataFrame to BigQuery
    - Empty the table beforehand if `truncate` is True, append otherwise
    """

    assert isinstance(data, pd.DataFrame)
    full_table_name = f"{gcp_project}.{bq_dataset}.{table}"
    #data.columns = ['pickup_datetime'] + [f"_{i}" for i in range(1,len(data.columns)-1)] + ['fare_amount']

    # Load data onto full_table_name
    client = bigquery.Client(gcp_project)
    write_mode = "WRITE_TRUNCATE" if truncate else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    job = client.load_table_from_dataframe(data, full_table_name, job_config=job_config)
    result = job.result()

    # 🎯 HINT for "*** TypeError: expected bytes, int found":
    # After preprocessing the data, your original column names are gone (print it to check),
    # so ensure that your column names are *strings* that start with either
    # a *letter* or an *underscore*, as BQ does not accept anything else


    print(f"✅ Data saved to bigquery, with shape {data.shape}")



In [13]:
full_table_name = f"{GCP_PROJECT}.{BQ_DATASET}.{BQ_REGION}"
full_table_name

'lw-bootcamp-429514.smallcapscout.EU'

In [14]:
load_data_to_bq(data=df, gcp_project=GCP_PROJECT, bq_dataset=BQ_DATASET, table=BQ_REGION, truncate=False)


✅ Data saved to bigquery, with shape (173345, 52)


In [15]:
ticker = 'META'
quarter = df.quarter.iloc[-1]
quarter

'2024-Q1'

In [16]:
query = f"""       
    SELECT *
    FROM {GCP_PROJECT}.{BQ_DATASET}.{BQ_REGION}
    WHERE TICKER = '{ticker}'
    ORDER BY DATE
    """

client = bigquery.Client(project=GCP_PROJECT)
query_job = client.query(query)
result = query_job.result()
data = result.to_dataframe()

/Users/christophernance/.pyenv/versions/3.10.6/envs/small-cap-scout/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [17]:
data

,name,TICKER,date,quarter,Assets,AssetsCurrent,Cash,AssetsNoncurrent,Liabilities,LiabilitiesCurrent,...,afs,sic_2d,GDP,interest_rate,unemployment_rate,median_cpi,small_cap,micro_cap,market_cap,qtr
0,"META PLATFORMS, INC.",META,2013-03-31,2013-Q1,1.516300e+10,1.104200e+10,2.325000e+09,4.121000e+09,3.339000e+09,1.063000e+09,...,4-NON,73,0.745356,0.14,7.6,1.215358,0,0,6.617569e+04,Q1
1,"META PLATFORMS, INC.",META,2013-06-30,2013-Q2,1.572400e+10,1.142100e+10,3.001000e+09,4.303000e+09,3.375000e+09,1.080000e+09,...,4-NON,73,2.083069,0.09,7.6,2.137250,0,0,5.887117e+04,Q2
2,"META PLATFORMS, INC.",META,2013-09-30,2013-Q3,1.493300e+10,1.054900e+10,3.100000e+09,4.384000e+09,1.885000e+09,9.840000e+08,...,4-NON,73,5.305288,0.08,7.2,2.232160,0,0,1.005641e+05,Q3
3,"META PLATFORMS, INC.",META,2013-12-31,2013-Q4,1.789500e+10,1.307000e+10,3.323000e+09,4.825000e+09,2.425000e+09,1.100000e+09,...,1-LAF,73,-2.479057,0.09,6.7,2.844644,0,0,1.154015e+05,Q4
4,"META PLATFORMS, INC.",META,2014-03-31,2014-Q1,1.902800e+10,1.406000e+10,2.998000e+09,4.968000e+09,2.291000e+09,1.037000e+09,...,1-LAF,73,4.429802,0.08,6.7,2.858977,0,0,1.745554e+05,Q1
5,"META PLATFORMS, INC.",META,2014-06-30,2014-Q2,2.076900e+10,1.555700e+10,4.384000e+09,5.212000e+09,2.423000e+09,1.214000e+09,...,1-LAF,73,6.147927,0.10,6.1,1.311364,0,0,1.624498e+05,Q2
6,"META PLATFORMS, INC.",META,2014-09-30,2014-Q3,2.418800e+10,1.611500e+10,8.999000e+09,8.073000e+09,2.950000e+09,1.234000e+09,...,1-LAF,73,1.806582,0.09,5.9,1.935411,0,0,1.945359e+05,Q3
7,"META PLATFORMS, INC.",META,2014-12-31,2014-Q4,4.018400e+10,1.367000e+10,4.315000e+09,2.651400e+10,4.088000e+09,1.424000e+09,...,1-LAF,73,4.056326,0.12,5.6,2.383394,0,0,2.093674e+05,Q4
8,"META PLATFORMS, INC.",META,2015-03-31,2015-Q1,4.200700e+10,1.500400e+10,3.419000e+09,2.700300e+10,4.506000e+09,1.746000e+09,...,1-LAF,73,3.978531,0.11,5.5,2.433336,0,0,2.210380e+05,Q1
9,"META PLATFORMS, INC.",META,2015-06-30,2015-Q2,4.413000e+10,1.695100e+10,5.123000e+09,2.717900e+10,4.678000e+09,1.881000e+09,...,1-LAF,73,1.737486,0.13,5.3,2.972400,0,0,2.254733e+05,Q2


In [27]:
quarter

'2024-Q1'

In [23]:
idx = data.index[data.quarter==quarter][0]
idx

44

In [33]:
np.array(input_data).reshape(1, input_data.shape[0], input_data.shape[1]).shape

(1, 4, 52)

In [26]:
input_data = data.iloc[(idx-(4-1)):idx+1]
input_data

,name,TICKER,date,quarter,Assets,AssetsCurrent,Cash,AssetsNoncurrent,Liabilities,LiabilitiesCurrent,...,afs,sic_2d,GDP,interest_rate,unemployment_rate,median_cpi,small_cap,micro_cap,market_cap,qtr
41,"META PLATFORMS, INC.",META,2023-06-30,2023-Q2,2.066880e+11,6.956000e+10,2.878500e+10,1.371280e+11,7.265500e+10,2.992100e+10,...,1-LAF,73,3.878915,5.08,3.6,4.396425,0,0,6.986264e+05,Q2
42,"META PLATFORMS, INC.",META,2023-09-30,2023-Q3,2.162740e+11,7.837800e+10,3.689000e+10,1.378960e+11,7.340100e+10,3.053100e+10,...,None,na,4.782863,5.33,3.8,5.217921,0,0,7.626336e+05,Q3
43,"META PLATFORMS, INC.",META,2023-12-31,2023-Q4,2.296230e+11,8.536500e+10,4.186200e+10,1.442580e+11,7.645500e+10,3.196000e+10,...,None,na,1.388811,5.33,3.7,4.160125,0,0,8.351953e+05,Q4
44,"META PLATFORMS, INC.",META,2024-03-31,2024-Q1,2.228440e+11,7.533000e+10,3.230700e+10,1.475140e+11,7.331500e+10,2.810100e+10,...,1-LAF,73,2.512104,5.33,3.8,4.327346,0,0,1.280566e+06,Q1


In [47]:
data.iloc[data.index[data.quarter==quarter]]

,TICKER,date,quarter,Assets,AssetsCurrent,Cash,AssetsNoncurrent,Liabilities,LiabilitiesCurrent,LiabilitiesNoncurrent,...,EffectOfExchangeRateFinal,CashPeriodIncreaseDecreaseIncludingExRateEffectFinal,afs,sic_2d,GDP,interest_rate,unemployment_rate,median_cpi,small_cap,micro_cap
55,AAPL,2023-12-31,2023-Q4,3.535140e+11,1.436920e+11,4.076000e+10,2.098220e+11,2.794140e+11,1.339730e+11,1.454410e+11,...,0.0,1.123700e+10,None,na,1.388811,5.33,3.7,4.160125,0,0


In [49]:
horizon = 'quarter-ahead'
if horizon not in ['quarter-ahead', 'year-ahead', 'two-years-ahead']:
    raise ValueError (f"Unsupported horizon: {horizon}, must be quarter-ahead, year-ahead or two-years-ahead")